In [0]:
def GE_calendar():
    """
    Great Expectations para Calendar.
    Expectativa para os tipos de dados das colunas.
    """

    # Calendar
    df_calendar = spark.read.parquet(path_silver_calendar)

    # Contextos
    context_calendar = gx.get_context()

    data_source_calendar = context_calendar.data_sources.add_spark("calendar_data_source")

    # Data Assets
    data_asset_calendar = data_source_calendar.add_dataframe_asset(name="calendar_data_asset")

    # Definição de Batches
    batch_definition_calendar = data_asset_calendar.add_batch_definition_whole_dataframe("calendar_batch_definition")

    # Batches
    batch_calendar = batch_definition_calendar.get_batch(batch_parameters={"dataframe": df_calendar})

    # Suite
    suite_calendar = gx.ExpectationSuite(name="suite_calendar")
    suite_calendar = context_calendar.suites.add(suite_calendar)

    # Colunas
    numericos_int = ["minimum_nights", "maximum_nights"]

    numericos_float = ["price", "adjusted_price"]

    numericos_long = ["listing_id"]

    colunas_data = ["date"]

    booleanos = ["available"]

    string_lista = [x for x in df_calendar.columns if x not in numericos_int + numericos_float + numericos_long + colunas_data + booleanos]

    # Validações
    for coluna in numericos_int:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="IntegerType")
        suite_calendar.add_expectation(expectation)

    for coluna in numericos_float:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="FloatType")
        suite_calendar.add_expectation(expectation)

    for coluna in numericos_long:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="LongType")
        suite_calendar.add_expectation(expectation)

    for coluna in colunas_data:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="DateType")
        suite_calendar.add_expectation(expectation)

    for coluna in booleanos:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="BooleanType")
        suite_calendar.add_expectation(expectation)

    for coluna in string_lista:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="StringType")
        suite_calendar.add_expectation(expectation)

    validation_definition_calendar = gx.ValidationDefinition(data=batch_definition_calendar, suite=suite_calendar, name="validacao_calendar")

    validation_definition_calendar.run(batch_parameters={"dataframe":df_calendar})
    

In [0]:
def GE_listings():
    """
    Great Expectations para Listings.
    Expectativa para os tipos de dados das colunas.
    Expectativa para algumas validações específicas.
    """

    # Listings
    df_listings = spark.read.parquet(path_silver_listings)

    # Contextos
    context_listings = gx.get_context()

    data_source_listings = context_listings.data_sources.add_spark("listings_data_source")

    # Data Assets
    data_asset_listings = data_source_listings.add_dataframe_asset(name="listings_data_asset")

    # Definição de Batches
    batch_definition_listings = data_asset_listings.add_batch_definition_whole_dataframe("listings_batch_definition")

    # Batches
    batch_listings = batch_definition_listings.get_batch(batch_parameters={"dataframe": df_listings})

    # Suite
    suite_listings = gx.ExpectationSuite(name="suite_listings")
    suite_listings = context_listings.suites.add(suite_listings)

    # Colunas

    numericos_int = ["host_id", "host_response_rate", "host_listings_count", "host_total_listings_count", 
                    "accommodates", "bedrooms", "beds", "minimum_nights", "maximum_nights", "minimum_minimum_nights", 
                    "maximum_minimum_nights", "minimum_maximum_nights", "maximum_maximum_nights", "availability_30", 
                    "availability_60", "availability_90", "availability_365", "number_of_reviews", "number_of_reviews_ltm", 
                    "number_of_reviews_l30d", "calculated_host_listings_count", "calculated_host_listings_count_entire_homes", 
                    "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms"]

    numericos_float = ["latitude", "longitude", "bathrooms", "price", "minimum_nights_avg_ntm", "maximum_nights_avg_ntm", 
                    "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", 
                    "review_scores_communication", "review_scores_location", "review_scores_value", "reviews_per_month"]

    numericos_long = ["id", "scrape_id"]

    colunas_data = ["last_scraped", "host_since", "calendar_last_scraped", "first_review", "last_review"]

    booleanos = ["host_is_superhost", "host_has_profile_pic", "host_identity_verified", "has_availability", "instant_bookable"]

    listas = ["host_verifications", "amenities"]

    string_lista = [x for x in df_listings.columns if x not in numericos_int + numericos_float + numericos_long + colunas_data + booleanos + listas]

    # Validações de Tipos
    for coluna in numericos_int:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="IntegerType")
        suite_listings.add_expectation(expectation)

    for coluna in numericos_float:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="FloatType")
        suite_listings.add_expectation(expectation)

    for coluna in numericos_long:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="LongType")
        suite_listings.add_expectation(expectation)

    for coluna in colunas_data:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="DateType")
        suite_listings.add_expectation(expectation)

    for coluna in booleanos:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="BooleanType")
        suite_listings.add_expectation(expectation)

    for coluna in listas:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="ListType")
        suite_listings.add_expectation(expectation)

    for coluna in string_lista:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="StringType")
        suite_listings.add_expectation(expectation)

    # Maior ou igual a 0
    for i in numericos_long + numericos_int + [x for x in numericos_float if x not in ["latitude", "longitude"]]:
        expectation = gx.expectations.ExpectColumnValuesToBeBetween(column=i, min_value=0, strict_min=True)
        suite_listings.add_expectation(expectation)

    # Rates 0-100
    for i in ["host_response_rate", "host_acceptance_rate"]:
        expectation = gx.expectations.ExpectColumnValueLengthsToBeBetween(column=i, min_value=0, max_value=100)
        suite_listings.add_expectation(expectation)

    # Rates 0-5
    for i in ["review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value", "reviews_per_month"]:
        expectation = gx.expectations.ExpectColumnValueLengthsToBeBetween(column=i, min_value=0, max_value=5)
        suite_listings.add_expectation(expectation)

    # Datas entre o ano de lançamento do AirBnB e a data atual
    today = date.today()
    air_bnb_release = date(2008,1,1)
    for i in colunas_data:
        expectation = gx.expectations.ExpectColumnValuesToBeBetween(column=i, min_value=air_bnb_release, max_value=today)
        suite_listings.add_expectation(expectation)

    # Valores Booleanos
    for i in booleanos:
        expectation = gx.expectations.ExpectColumnValuesToBeInSet(column=i, value_set=[True, False])
        suite_listings.add_expectation(expectation)

    # Latitude
    expectation = gx.expectations.ExpectColumnValueLengthsToBeBetween(column="latitude", min_value=-90, max_value=90)
    suite_listings.add_expectation(expectation)

    # Longitude
    expectation = gx.expectations.ExpectColumnValueLengthsToBeBetween(column="longitude", min_value=-180, max_value=180)
    suite_listings.add_expectation(expectation)

    validation_definition_listings = gx.ValidationDefinition(data=batch_definition_listings, suite=suite_listings, name="validacao_listings")

    validation = validation_definition_listings.run(batch_parameters={"dataframe":df_listings})

    return validation

In [0]:
def GE_reviews():
    """
    Great Expectations para Reviews.
    Expectativa para os tipos de dados das colunas.
    """
    
    # Reviews
    df_reviews = spark.read.parquet(path_silver_reviews)

    # Contextos
    context_reviews = gx.get_context()

    data_source_reviews = context_reviews.data_sources.add_spark("reviews_data_source")

    # Data Assets
    data_asset_reviews = data_source_reviews.add_dataframe_asset(name="reviews_data_asset")

    # Definição de Batches
    batch_definition_reviews = data_asset_reviews.add_batch_definition_whole_dataframe("reviews_batch_definition")

    # Batches
    batch_reviews = batch_definition_reviews.get_batch(batch_parameters={"dataframe": df_reviews})

    # Suite
    suite_reviews = gx.ExpectationSuite(name="suite_reviews")
    suite_reviews = context_reviews.suites.add(suite_reviews)

    # Colunas
    numericos_long = ["listing_id", "id", "reviewer_id"]

    colunas_data = ["date"]

    string_lista = [x for x in df_reviews.columns if x not in numericos_long + colunas_data]

    # Validações
    for coluna in numericos_int:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="IntegerType")
        suite_reviews.add_expectation(expectation)

    for coluna in numericos_float:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="FloatType")
        suite_reviews.add_expectation(expectation)

    for coluna in numericos_long:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="LongType")
        suite_reviews.add_expectation(expectation)

    for coluna in colunas_data:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="DateType")
        suite_reviews.add_expectation(expectation)

    for coluna in booleanos:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="BooleanType")
        suite_reviews.add_expectation(expectation)

    for coluna in string_lista:
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="StringType")
        suite_reviews.add_expectation(expectation)

    validation_definition_reviews = gx.ValidationDefinition(data=batch_definition_reviews, suite=suite_reviews, name="validacao_reviews")

    validation_definition_reviews.run(batch_parameters={"dataframe":df_reviews})